The meaning of words often changes over time.  In this homework, you will explore this phenomenon by identifying shifts in word meaning over the space of one hundred years by examining word embeddings trained on historical data (largely published before 1923) and those trained on contemporary texts.

In [311]:
import re
from gensim.models import KeyedVectors
import operator

In [312]:
wiki = KeyedVectors.load_word2vec_format("../data/glove.6B.50d.50K.txt", binary=False)

In [313]:
guten = KeyedVectors.load_word2vec_format("../data/gutenberg.200.vectors.50K.txt", binary=False)

Q1. Before we jump in, select 5 words whose senses you believe have changed over the period of the past 100 years. Ensure they are in the vocabulary of both models.  Explain the two different meanings they have.  This is an important step in stating your beliefs before you examine any empirical evidence; do not change these terms after you have run the models you develop below.  (Here we are only evaluating the rationales, not whether the terms *actually* undergo sense change, as measured below.)

In [314]:
# fill in terms here
terms=["gay", "fanny", "flirt", "sick", "cell"]
for term in terms:
    if term not in wiki or term not in guten:
        print("%s missing!" % term)

*Gay*: Gay used to mean happy, debonair, or stylish. It now most commonly used to refer to folks who are attracted to the same sex (or more expansively: people who are not heterosexual).

*Fanny*: Fanny used to explicitly refer to a vagina. In the States, it now colloquially refers to someone's butt or a "fanny pack."

*Flirt*: Flirt now means to try and woo. It used to mean to flick or to scorn.

*Sick*: Sick used to mean that you were ill. Now it can mean that, but it can also mean that something is cool or "radical, dude!"

*Cell*: Cell can refer to a small room or a jail cell. Now I believe it often refers to cell phone.

Q2. Find the words that have changed the most by calculating the number of words that overlap in their 50 nearest neighbors.  That is, let $\mathcal{N}_{guten}(\textrm{awesome})$ be the 50 nearest neighbors for the word "awesome" in the Gutenberg embeddings and $\mathcal{N}_{wiki}(\textrm{awesome})$ be the 50 nearest neighbors for "awesome" in the Wikipedia embeddings.  Calculate the size of $\mathcal{N}_{guten}(\textrm{awesome}) \cap \mathcal{N}_{wiki}(\textrm{awesome})$.  Under this method, the words that share the *fewest* neighbors have moved the furthest apart.  Display the 100 words that have moved the furthest apart and the 100 words that have remained the closest together, along with their intersection score.  

In [315]:
# Find 50 nearest neighbors of words in Gutenberg and Wikipedia
def print_top(word):
    print("=== %s ===\n" % word)
    print("Gutenberg:")
    for k, v in guten.most_similar(word, topn=50):
        print("%.3f\t%s" % (v,k))

    print()
    print("Wikipedia:")
    for k, v in wiki.most_similar(word, topn=50):
        print("%.3f\t%s" % (v,k)) 
    print()

for term in terms:
    print_top(term)

=== gay ===

Gutenberg:
0.678	merry
0.659	joyous
0.645	lively
0.622	debonair
0.613	gayest
0.609	light-hearted
0.606	vivacious
0.606	cheerful
0.599	blithe
0.593	carefree
0.590	rollicking
0.576	charming
0.572	gaiety
0.571	gayety
0.570	sprightly
0.568	watteau
0.566	brilliant
0.565	pleasant
0.565	lightsome
0.561	care-free
0.558	mirthful
0.554	lighthearted
0.551	sociable
0.550	airy
0.549	demure
0.548	well-mannered
0.548	rosy-cheeked
0.546	quaintest
0.546	happy
0.546	toilettes
0.544	exuberant
0.543	chatty
0.543	gayer
0.542	trig
0.542	good-humoured
0.540	ravishing
0.539	jovial
0.538	hoyden
0.538	boyish
0.535	parades
0.535	captivating
0.534	homelike
0.534	smart
0.534	showy
0.533	hilarious
0.533	jolly
0.532	blooming
0.531	colorful
0.531	festal
0.530	pleasing

Wikipedia:
0.850	lesbian
0.801	abortion
0.801	gays
0.799	lesbians
0.790	homosexual
0.765	homosexuals
0.763	sex
0.725	bisexual
0.707	lgbt
0.699	transgender
0.692	homosexuality
0.688	openly
0.674	bashing
0.672	rights
0.664	advocates
0.647	ha

In [316]:
import numpy as np
guten_sick = guten.most_similar("sick", topn = 50)
wiki_sick = wiki.most_similar("sick", topn = 50)
sick_intersect = np.intersect1d(guten_sick, wiki_sick)

In [317]:
sick_intersect
len(list(sick_intersect))

7

In [318]:
def intersect_term(word):
    guten_term = guten.most_similar(word, topn = 50)
    wiki_term = wiki.most_similar(word, topn = 50)
    term_intersect = len(np.intersect1d(guten_term, wiki_term))
    
    return(term_intersect)

**Q2 part 1 response:**

In [319]:
import pandas as pd
# I do this for each of the five words that I defined as interesting above.
overlap = []
for term in terms:
    
    term_list = terms
    
    overlap.append(intersect_term(term))

# OUTSIDE FOR LOOP

print(term_list)
print(overlap)

pd.DataFrame(
    {'term': term_list,
     'overlap': overlap
    })
    
# The word that shares the fewest neighbors between Gutenberg and Wiki is gay (0 neighbors)
# The word that shares the most is sick (7 neighbors)

['gay', 'fanny', 'flirt', 'sick', 'cell']
[0, 4, 3, 7, 1]


,term,overlap
0,gay,0
1,fanny,4
2,flirt,3
3,sick,7
4,cell,1


**Q2: part 2 response** 
    Under this method, the words that share the fewest neighbors have moved the furthest apart. 
    Display the 100 words that have moved the furthest apart and the 100 words that 
    have remained the closest together, along with their intersection score.

In [320]:
# Let's find all the overlapping words between Guten and Wiki
# print(guten)
# print(wiki)

guten_words = guten.key_to_index.keys()
wiki_words = wiki.key_to_index.keys()
dict_intersect = list(np.intersect1d(list(guten_words), list(wiki_words)))

# dict_intersect = dict_intersect[:10] # testing with first 10 elements
# dict_intersect

# len(dict_intersect)
# There are 26509 shared words between guten_words and dict_intersect

In [321]:
overlap = []

for term in dict_intersect:
    guten_term = guten.most_similar(term, topn = 50)
    wiki_term = wiki.most_similar(term, topn = 50)
    term_intersect = len(list(np.intersect1d(guten_term, wiki_term)))

    term_list = dict_intersect
    
    overlap.append(intersect_term(term))

# OUTSIDE FOR LOOP

# print(term_list)
# print(overlap)

df_dict = pd.DataFrame(
    {'term': term_list,
     'overlap': overlap
    })



In [322]:
pd.set_option("display.max_rows", 100, "display.max_columns", 100)
    # Doing this to display the words asked for

# Need to make sure overlap column is numeric
df_dict["overlap"] = df_dict["overlap"].astype(float)
# Need to sort the dataframe
df_dict.sort_values(by=["overlap"], ascending = True, inplace = True)

# print(df_dict)

# Show first 100 rows (words that have moved furthest apart --- least neighbors)
    # Problem: There may be more than 100 words with 0 overlap between the two dictionaries
df_dict.head(100)["term"]




13254           ishmael
7486               ding
7481              dinah
7480                din
7479            dimming
17948          pitchers
17950         pitchfork
17952          pitfalls
17954            pitman
7469          dimension
7468               dime
17956              pitt
7464               dill
17958           pitting
17962              pius
7457               dike
7456              dijon
7455               digs
7410        dictatorial
7411       dictatorship
18038          pleading
18037           pleaded
18027        playground
18026         playfully
17944          pitcairn
18015             platt
7429      differentiate
18003           plastic
17984            planes
7442             digest
7443           digested
17975           plagues
7423            dietary
7408          dictating
17936            piquet
17934            piping
17875              piet
7540      disappointing
17884              pike
17893        pilkington
17901             pills
17903           

In [323]:
# Let's filter the dataset to words with 0 overlap
print(df_dict.loc[df_dict["overlap"] == 0])

# There are 6,832 rows with 0 overlap. This means 6,832 words share no overlapping neighbors (at least the 50 closest),
# though they are present in the Gutenberg and Wikipedia

               term  overlap
13254       ishmael      0.0
7486           ding      0.0
7481          dinah      0.0
7480            din      0.0
7479        dimming      0.0
...             ...      ...
23173      suleiman      0.0
25335  verification      0.0
21619        shrimp      0.0
21613      shredded      0.0
23718      tendered      0.0

[6832 rows x 2 columns]


In [324]:
# Show last 100 rows (words that have stayed closest together --- most neighbors)
df_dict[-100:]["term"]

# There are terms like "9th" and numbers in the provided dictionaries. I've kept them for completeness.
# It makes sense that these "words" have a lot of overlap --- our use of numbers has remained similar over
# time.

419              20
24649        twelve
350            1878
158            16th
338            1867
126            15th
111              15
803              51
919             8th
335            1864
852             5th
110            14th
789             4th
342            1870
263              18
305            1837
307            1839
932             9th
348            1876
380            1903
35                1
358            1885
378            1901
684              40
564               3
13663      kentucky
23903    thirteenth
382            1905
97             13th
877              63
13204          iowa
85             12th
828              54
159              17
10277      fourteen
375              19
86               13
59               11
332            1861
385            1908
329            1859
902              77
296            1829
127              16
580              31
325            1855
452              21
373            1899
299            1831
309            1840


Now let's look at how much the candidate terms you defined above have changed their meaning as measured in these embeddings.  First, we can just print their neighborhoods:

In [325]:
def print_top(word):
    print("=== %s ===\n" % word)
    print("Gutenberg:")
    for k, v in guten.most_similar(word, topn=10):
        print("%.3f\t%s" % (v,k))

    print()
    print("Wikipedia:")
    for k, v in wiki.most_similar(word, topn=10):
        print("%.3f\t%s" % (v,k)) 
    print()

In [326]:
for term in terms:
    print_top(term)

=== gay ===

Gutenberg:
0.678	merry
0.659	joyous
0.645	lively
0.622	debonair
0.613	gayest
0.609	light-hearted
0.606	vivacious
0.606	cheerful
0.599	blithe
0.593	carefree

Wikipedia:
0.850	lesbian
0.801	abortion
0.801	gays
0.799	lesbians
0.790	homosexual
0.765	homosexuals
0.763	sex
0.725	bisexual
0.707	lgbt
0.699	transgender

=== fanny ===

Gutenberg:
0.686	gertrude
0.653	elizabeth
0.653	charlotte
0.648	clara
0.644	ruth
0.641	annie
0.634	carrie
0.629	milicent
0.626	maggie
0.625	marjory

Wikipedia:
0.737	matilda
0.655	marguerite
0.647	aunt
0.642	sister-in-law
0.638	mistress
0.634	jacqueline
0.632	bessie
0.627	ella
0.626	isabella
0.625	antoinette

=== flirt ===

Gutenberg:
0.614	flirting
0.589	portionless
0.576	tease
0.569	flirtation
0.567	jilt
0.547	grumpy
0.546	coquet
0.545	minx
0.543	flirted
0.539	spitfire

Wikipedia:
0.745	indulge
0.693	giggling
0.692	tease
0.675	flirting
0.670	gravitate
0.664	hating
0.654	bored
0.650	giggle
0.647	madly
0.641	amuse

=== sick ===

Gutenberg:
0.668	tired

**check+**. Let's make this a little more precise.  Rank all terms by the overlap score you created above, so that words with scores closer to 0 (i.e., no overlap in nearest neighbors) are ranked higher (i.e., closer to position 1). Measure how good your guesses were by calculating their [mean reciprocal rank](https://en.wikipedia.org/wiki/Mean_reciprocal_rank) within this list.  (Again, we're not evaluating how good your guesses were above, but rather the correctness of your implementation of MRR.)

In [327]:
# Rank all terms by the overlap in the score created above. 
# Words w/ scores closer to 0 (no overlap w/ nearest neighbors) are ranked higher (closer to position 1).
# Measure how good your guesses were by calculating: MEAN RECIPROCAL RANK.


